# This notebook will implement an optional pre-processing step that can be done prior to candidate label creation.
We will trim down a given quantity file by using associated qualifiers to keep only the most up-to-date value for time-series data. For example, if your quantity file includes the human development index (HDI) of various countries, you may find that there are many values of HDI for each country. Trimming these values by keeping only the most recent will mitigate confusing results (e.g. labeling all countries as having a small population since all countries at some point had a smaller population) and reduce the number of candidate labels we create in later steps.

In [43]:
import os
import pandas as pd
import numpy as np
from utility import run_command
from utility import rename_cols_and_overwrite_id

### Parameters
**Required**   
*quantity_file*: file path for the file that contains entity to quantity-type values  
*qualifiers_file*: file path for the file that contains wikidata labels  
*out_file*: file path for the file that we will save the trimmed results in  
*work_dir*: path to folder where intermediate files created by this notebook should be saved.  
*store_dir*: path to folder containing the sqlite3.db file that we will use for our queries. We will reuse an existing file if there is one in this folder. Otherwise we will create a new one.

In [32]:
# **REQUIRED**
quantity_file = "../../Q154/data/parts/claims.quantity.tsv.gz"
qualifiers_file = "../../Q154/data/parts/qualifiers.quantity.tsv.gz"
out_file = "../../Q154/data/parts/claims.quantity_trimmed.tsv.gz"
work_dir = "../../Q154/profiler_work"
store_dir = "../../Q154"

### Process parameters and set up variables / file names

In [33]:
# Ensure paths are absolute
quantity_file = os.path.abspath(quantity_file)
qualifiers_file = os.path.abspath(qualifiers_file)
out_file = os.path.abspath(out_file)
work_dir = os.path.abspath(work_dir)
store_dir = os.path.abspath(store_dir)
    
# Create directories
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
work_dir = "{}/trim_quantity_work".format(work_dir)
if not os.path.exists(work_dir):
    os.makedirs(work_dir)

# Environment variables for kgtk commands
os.environ['QUANTITY_FILE'] = quantity_file
os.environ['QUALIFIERS_FILE'] = qualifiers_file
os.environ['WORK'] = work_dir
os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(store_dir)
os.environ['OUT_FILE'] = out_file
os.environ['kgtk'] = "kgtk" # Need to do this for kgtk to be recognized as a command when passing it through a subprocess call

See contents of qualifiers and quantity files below:

In [28]:
!gzcat $QUALIFIERS_FILE | head -5 | column -t -s $'\t'

gzcat: error writing to output: Broken pipe
gzcat: /Users/nicklein/Documents/grad_school/Research.nosync/Q154/data/parts/qualifiers.quantity.tsv.gz: uncompress failed
id                                           node1                          label  node2
Q1000-P1081-0d345f-3a33abf5-0-P585-90ede1-0  Q1000-P1081-0d345f-3a33abf5-0  P585   ^2004-01-01T00:00:00Z/9
Q1000-P1081-0d345f-6da37c02-0-P585-e4bef8-0  Q1000-P1081-0d345f-6da37c02-0  P585   ^2003-01-01T00:00:00Z/9
Q1000-P1081-1100e3-c7631769-0-P585-c03b8d-0  Q1000-P1081-1100e3-c7631769-0  P585   ^1992-01-01T00:00:00Z/9
Q1000-P1081-1ada51-7c71c229-0-P585-7131d5-0  Q1000-P1081-1ada51-7c71c229-0  P585   ^2002-01-01T00:00:00Z/9


In [30]:
!gzcat $QUANTITY_FILE | head -5 | column -t -s $'\t'

gzcat: error writing to output: Broken pipe
gzcat: /Users/nicklein/Documents/grad_school/Research.nosync/Q154/data/parts/claims.quantity.tsv.gz: uncompress failed
id                             node1  label  node2   node2;wikidatatype
Q1000-P1081-0d345f-3a33abf5-0  Q1000  P1081  +0.641  quantity
Q1000-P1081-0d345f-6da37c02-0  Q1000  P1081  +0.641  quantity
Q1000-P1081-1100e3-c7631769-0  Q1000  P1081  +0.624  quantity
Q1000-P1081-1ada51-7c71c229-0  Q1000  P1081  +0.639  quantity


Now using the quantity ID column and qualifiers node1 column to match quantity edges with P585 ('point in time') qualifiers.

In [35]:
!kgtk query --graph-cache $STORE \
-i $QUANTITY_FILE -i $QUALIFIERS_FILE -o $WORK/quantities_with_qualifiers.tsv.gz \
--match '`'"$QUANTITY_FILE"'`: (n1)-[l {label:prop}]->(n2), `'"$QUALIFIERS_FILE"'`: (l)-[q {label:P585}]->(t)' \
--return 'distinct n1, prop as label, n2 as node2, q.label as qualifier, kgtk_date_and_time(t) as time, l as id' \
--order-by 'n1, prop, q.label, time desc'

In [64]:
!gzcat $WORK/quantities_with_qualifiers.tsv.gz | head | column -t -s $'\t'

gzcat: error writing to output: Broken pipe
gzcat: /Users/nicklein/Documents/grad_school/Research.nosync/Q154/profiler_work/trim_quantity_work/quantities_with_qualifiers.tsv.gz: uncompress failed
node1  label  node2   qualifier  time                   id
Q1000  P1081  +0.702  P585       ^2017-01-01T00:00:00Z  Q1000-P1081-345681-88a99cab-0
Q1000  P1081  +0.698  P585       ^2016-01-01T00:00:00Z  Q1000-P1081-ca6790-6e605c9e-0
Q1000  P1081  +0.694  P585       ^2015-01-01T00:00:00Z  Q1000-P1081-81998e-cc0171b8-0
Q1000  P1081  +0.693  P585       ^2014-01-01T00:00:00Z  Q1000-P1081-70c922-1b54e63b-0
Q1000  P1081  +0.684  P585       ^2014-00-00T00:00:00Z  Q1000-P1081-703676-2421fe86-0
Q1000  P1081  +0.687  P585       ^2013-01-01T00:00:00Z  Q1000-P1081-f9db15-49b5f867-0
Q1000  P1081  +0.679  P585       ^2013-00-00T00:00:00Z  Q1000-P1081-eec082-fcd7ee29-0
Q1000  P1081  +0.678  P585       ^2012-01-01T00:00:00Z  Q1000-P1081-85a71b-cef7b7e5-0
Q1000  P1081  +0.673  P585       ^2012-00-00T00:00:00Z  Q

Next, remove the most recent edge for each <node1, label> pair

In [62]:
def mask_first(x):
    result = np.ones_like(x)
    result[0] = 0
    return result
df = pd.read_csv("{}/quantities_with_qualifiers.tsv.gz".format(os.environ["WORK"]), delimiter = '\t').fillna("")
mask = df.groupby(['node1','label'])["node1"].transform(mask_first).astype(bool)
out_file = "{}/quantity_edges_to_filter.tsv.gz".format(os.environ["WORK"])
df.loc[mask].to_csv(out_file, sep='\t', index = False)

See that the first edge for node1=Q1000, label=P1081 has been removed in this new file:

In [66]:
!gzcat $WORK/quantity_edges_to_filter.tsv.gz | head | column -t -s $'\t'

gzcat: error writing to output: Broken pipe
gzcat: /Users/nicklein/Documents/grad_school/Research.nosync/Q154/profiler_work/trim_quantity_work/quantity_edges_to_filter.tsv.gz: uncompress failed
node1  label  node2   qualifier  time                   id
Q1000  P1081  +0.698  P585       ^2016-01-01T00:00:00Z  Q1000-P1081-ca6790-6e605c9e-0
Q1000  P1081  +0.694  P585       ^2015-01-01T00:00:00Z  Q1000-P1081-81998e-cc0171b8-0
Q1000  P1081  +0.693  P585       ^2014-01-01T00:00:00Z  Q1000-P1081-70c922-1b54e63b-0
Q1000  P1081  +0.684  P585       ^2014-00-00T00:00:00Z  Q1000-P1081-703676-2421fe86-0
Q1000  P1081  +0.687  P585       ^2013-01-01T00:00:00Z  Q1000-P1081-f9db15-49b5f867-0
Q1000  P1081  +0.679  P585       ^2013-00-00T00:00:00Z  Q1000-P1081-eec082-fcd7ee29-0
Q1000  P1081  +0.678  P585       ^2012-01-01T00:00:00Z  Q1000-P1081-85a71b-cef7b7e5-0
Q1000  P1081  +0.673  P585       ^2012-00-00T00:00:00Z  Q1000-P1081-563843-b7b88c66-0
Q1000  P1081  +0.670  P585       ^2011-01-01T00:00:00Z  Q10

Now remove edges from the original quantity file if they are present in the edges-to-filter file. This will give us the final trimmed quantity file.

In [68]:
!kgtk ifnotexists -i $QUANTITY_FILE --filter-on $WORK/quantity_edges_to_filter.tsv.gz -o $OUT_FILE \
--input-keys id --filter-keys id

In [74]:
!gzcat $OUT_FILE | head | column -t -s $'\t'

id                             node1  label  node2                            node2;wikidatatype
Q1000-P1081-345681-88a99cab-0  Q1000  P1081  +0.702                           quantity
Q1000-P1082-195d9d-89877729-0  Q1000  P1082  +2025137                         quantity
Q1000-P1198-cbe76f-257c1a22-0  Q1000  P1198  +20[+19,+21]Q11229               quantity
Q1000-P1279-35bdb9-246dd402-0  Q1000  P1279  +4.1[+4.0,+4.2]Q11229            quantity
Q1000-P2046-66ad5c-c937c386-0  Q1000  P2046  +267667[+267666,+267668]Q712226  quantity
Q1000-P2131-b51f85-6e5e6371-0  Q1000  P2131  +14622880885.6842Q4917           quantity
Q1000-P2132-2acedd-b09a609d-0  Q1000  P2132  +7413Q4917                       quantity
Q1000-P2134-8bcbd1-3c5aea8e-0  Q1000  P2134  +965054219Q4917                  quantity
Q1000-P2219-8df143-a1b69147-0  Q1000  P2219  +2.3[+2.2,+2.4]Q11229            quantity
gzcat: error writing to output: Broken pipe
gzcat: /Users/nicklein/Documents/grad_school/Research.nosync/Q154/data/parts